In [8]:
import openpyxl

class Conversation:
    def __init__(self, contentName, contentType, question, answer):
        self.contentName = contentName
        self.contentType = contentType
        self.question = question
        self.answer = answer

    def __str__(self):
        return "질문 : " + self.question + "\n답변 : " + self.answer + "\n"
# 영어 대화 데이터가 담긴 엑셀 파일을 연다.

wb = openpyxl.load_workbook("input.xlsx")

ws = wb.active

conversations = []

# 시트 내에 존재하는 모든 영어 데이터를 객체로 담음
for r in ws.rows:
    c = Conversation(r[0].value,r[1].value,r[2].value,r[3].value)
    conversations.append(c)

wb.close()

for c in conversations:
    print(str(c))


질문 : How many people are there in your family?
답변 : There are 5 people.

질문 : Does your family live in a house or an apartment?
답변 : We live in a house in the countryside



In [10]:
# -*- coding: utf-8 -*-



# 파일로 출력하기

i = 1

# 출력, 입력 값 JSON 파일을 생성합니다.

prev = str(conversations[0].contentName) + str(conversations[0].contentType)

f = open(prev + '.json', 'w', encoding='UTF-8')

f.write('{ "id": "10d3155d-4468-4118-8f5d-15009af446d0", "name": "' + prev + '", "auto": true, "contexts": [], "responses": [ { "resetContexts": false, "affectedContexts": [], "parameters": [], "messages": [ { "type": 0, "lang": "ko", "speech": "' + conversations[0].answer + '" } ], "defaultResponsePlatforms": {}, "speech": [] } ], "priority": 500000, "webhookUsed": false, "webhookForSlotFilling": false, "fallbackIntent": false, "events": [] }')

f.close()

f = open(prev + '_usersays_ko.json', 'w', encoding='UTF-8')

f.write("[")

f.write('{ "id": "3330d5a3-f38e-48fd-a3e6-000000000001", "data": [ { "text": "' + conversations[0].question + '", "userDefined": false } ], "isTemplate": false, "count": 0 },')



while True:

    if i >= len(conversations):

        f.write("]")

        f.close()

        break;

    c = conversations[i]

    if prev == str(c.contentName) + str(c.contentType):

        f.write('{ "id": "3330d5a3-f38e-48fd-a3e6-000000000001", "data": [ { "text": "' + c.question + '", "userDefined": false } ], "isTemplate": false, "count": 0 },')

    else:

        f.write("]")

        f.close()

        # 출력, 입력 값 JSON 파일을 생성합니다.

        prev = str(c.contentName) + str(c.contentType)

        f = open(prev + '.json', 'w', encoding='UTF-8')

        f.write('{ "id": "10d3155d-4468-4118-8f5d-15009af446d0", "name": "' + prev + '", "auto": true, "contexts": [], "responses": [ { "resetContexts": false, "affectedContexts": [], "parameters": [], "messages": [ { "type": 0, "lang": "ko", "speech": "' + c.answer + '" } ], "defaultResponsePlatforms": {}, "speech": [] } ], "priority": 500000, "webhookUsed": false, "webhookForSlotFilling": false, "fallbackIntent": false, "events": [] }')

        f.close()

        f = open(prev + '_usersays_ko.json', 'w', encoding='UTF-8')

        f.write("[")

        f.write('{ "id": "3330d5a3-f38e-48fd-a3e6-000000000001", "data": [ { "text": "' + c.question + '", "userDefined": false } ], "isTemplate": false, "count": 0 },')

    i = i + 1


In [11]:
# -*- encoding: utf-8 -*-

import requests

import json



from flask import Flask, request, jsonify



def get_answer(text, user_key):

    

    data_send = { 

        'query': text,

        'sessionId': user_key,

        'lang': 'ko',

    }

    

    data_header = {

        'Authorization': '964868a4906b6253f5f3ef87b3efc6e64942bb25',

        'Content-Type': 'application/json; charset=utf-8'

    }

    

    dialogflow_url = 'https://api.dialogflow.com/v1/query?v=20150910'

    

    res = requests.post(dialogflow_url, data=json.dumps(data_send), headers=data_header)



    if res.status_code != requests.codes.ok:

        return '오류가 발생했습니다.'

    

    data_receive = res.json()

    answer = data_receive['result']['fulfillment']['speech'] 

    

    return answer



app = Flask(__name__)

app.config['JSON_AS_ASCII'] = False



@app.route('/', methods=['POST', 'GET'])

def webhook():



    content = request.args.get('content')

    userid = request.args.get('userid')



    return get_answer(content, userid)



if __name__ == '__main__':

    app.run(host='0.0.0.0')    



출처: https://ndb796.tistory.com/133?category=1013435 [안경잡이개발자]

ModuleNotFoundError: No module named 'dialogflow_v2beta1'